In [1]:
import sys
sys.path.append('../util/')
sys.path.append('../datasets/')
sys.path.append('../models/')

import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
import numpy as np
from sklearn import model_selection
from scipy.optimize import fmin_cg
import scipy.signal
import random
import matplotlib.pyplot as plt
from typing import *

from engine import setup_seed,Namespace,train_vae_one_epoch_test
from datasets import build_dataset
from models import build_models
from monitor import Monitor


In [2]:
setup_seed(42)

In [3]:


args = Namespace()

args.dataset_type ='cylinder2d-p'
args.dataset_path =  r'../Sea_Surface_Temperature.npy'
args.dataset_mask_path =  r'../Sea_Surface_Temperature_mask.npy'
args.test_size = 0.33

args.model_type = "MLP"
args.mod_number = 32
args.pod_loss = None

args.lr = 1e-4
args.step_size = 20
args.gamma = 0.1
args.epochs=int(50)

args.log_interval = 100
args.batch_size = 10
args.device='cuda:0'
args.word_dir='work_dir/'


args.monitorType = "random,sea"
# args.monitorGridShape = (5,5)
args.random_num = 25

# log

In [4]:
localtime = time.asctime( time.localtime(time.time()) )
localtime = localtime.replace(' ','_')
localtime = localtime.replace(':','_')
print(localtime)
log_path = './log/'+localtime

os.makedirs(log_path)


    
with open(log_path+'/arg.txt','w',encoding='utf-8') as f:
    f.write(str(vars(args)))

Fri_Mar_24_13_06_03_2023


# dataset

In [5]:
build_dataset_res= build_dataset(args.dataset_path,args.dataset_type,args.test_size,)
train_dataset = build_dataset_res['train_dataset']
val_dataset  = build_dataset_res['val_dataset']

TM =  train_dataset.data.mean(axis=0)
train_dataset.data = train_dataset.data - train_dataset.data.mean(axis=0)

args.data_shape =  build_dataset_res['data_shape']

print("val_dataset.shape",val_dataset.shape)
print("train_dataset.shape",train_dataset.shape)

train_loader = DataLoader(train_dataset,batch_size=args.batch_size)
val_loader = DataLoader(val_dataset,batch_size=1)

val_dataset.shape (180, 360)
train_dataset.shape (180, 360)


# module

In [6]:
build_model = build_models(
    args.model_type,
    args.data_shape,
    args.mod_number,
    args.pod_loss
)
model = build_model['model']
args.mod_input_shape=build_model['mod_input_shape']
args.mod_output_shape=build_model['mod_output_shape']
args.code_shape=build_model['code_shape']


with open(log_path+'/model.txt','w',encoding='utf-8') as f:
    f.write(str(model))
    

# train pipeline

In [7]:
optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode='min', factor=args.gamma, patience=args.step_size, threshold=0.00001,)
loss = nn.MSELoss()

# train fun

In [8]:
def train_vae_one_epoch_test(
        model: torch.nn.Module,
        train_loader, 
        optimizer: torch.optim.Optimizer,
        device: torch.device, 
        epoch: int, 
        mod_input_shape,
        mod_output_shape,
        log_interval,
        ):
    
    model.train()
    for batch_idx, (data) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        B_size = len(data)
        B_mod_input_shape = [B_size,]+ list(mod_input_shape)
        B_mod_output_shape = [B_size,]+ list(mod_output_shape)
        output = model(data.reshape(B_mod_input_shape))
        loss_info = model.loss_function(data.reshape(B_mod_output_shape),output)
        loss = loss_info['loss']
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# val fun

In [9]:
target = val_dataset[15]-TM
SSTmask = np.load(args.dataset_mask_path) 
"""
Monitor(self,monitorType ,data_shape,
        random_num=None, monitorGridShape=None
"""
meansure = Monitor(args.monitorType, args.data_shape,random_num = args.random_num ,mask = np.load(args.dataset_mask_path)  )
mask_map = meansure.grid2D()
mask_map = torch.Tensor(mask_map)

    
def val(targets : List ,m_s  : List ,model,code_shape,output_shape,device):
    con_m=[]
    for m in m_s:
        res = []
        for target in targets:
            model = model.to(device)
            mask_map = m.grid2D()
            mask_map = torch.Tensor(mask_map)

            target_re = target.reshape(output_shape).to(device)
            mask_re   = mask_map.reshape(output_shape).to(device)

            f = model.loss_decoder_helper(target_re,mask_re)
            fp= model.grad_loss_decoder_helper(target_re,mask_re)

            start_code = np.zeros(args.mod_number).astype(np.float32)+0.1
            fmin_code =fmin_cg(f,start_code,fprime=fp,disp=False )
            B_code_shape = [1,]+list(code_shape)
            y_per = model.decode( torch.Tensor(fmin_code.astype(np.float32)).to(device).reshape(B_code_shape))

            DS = (target.flatten().detach().cpu()-y_per.flatten().detach().cpu())
            D = torch.sqrt((DS**2).sum())
            T = torch.sqrt(((target.detach()+TM)**2).sum())
            L2 = D/T
            L2 = L2.cpu().numpy()
            
            res.append(L2)
        res = np.array(res ).mean()
        con_m.append(res)
    return con_m 


# train

In [16]:
model.to(args.device)

val_best = np.inf

for epoch in range(1, args.epochs+ 1):
    model = model.to(args.device)
    train_vae_one_epoch_test( 
        model,  
        train_loader, 
        optimizer, 
        args.device, 
        epoch ,
        args.mod_input_shape,
        args.mod_output_shape,
        args.log_interval
    )


    if epoch % 10 == 0 and epoch>1:
        val_res = val([target,],[meansure,],model,args.code_shape,args.mod_output_shape,args.device)
        #def val(targets,m_s,model,code_shape,output_shape,device):
        if val_best > val_res[0]:
            val_best = val_res[0]
            torch.save(model,log_path+'/MLP32-SST-best.pth')
            torch.save(model,'./log/MLP32-SST-best-%f.pth'%(val_best))
            print('torch.save ','./log/MLP32-SST-best-%f.pth'%(val_best))
            
        print(val_res)

torch.save(model,log_path+'/last.pth')


Train Epoch: 1 [0/1062 (0%)]	Loss: 0.162871
Train Epoch: 1 [1000/1062 (93%)]	Loss: 0.136621
Train Epoch: 2 [0/1062 (0%)]	Loss: 0.161795
Train Epoch: 2 [1000/1062 (93%)]	Loss: 0.136848
Train Epoch: 3 [0/1062 (0%)]	Loss: 0.165830
Train Epoch: 3 [1000/1062 (93%)]	Loss: 0.137460
Train Epoch: 4 [0/1062 (0%)]	Loss: 0.162161
Train Epoch: 4 [1000/1062 (93%)]	Loss: 0.136596
Train Epoch: 5 [0/1062 (0%)]	Loss: 0.155549
Train Epoch: 5 [1000/1062 (93%)]	Loss: 0.143272
Train Epoch: 6 [0/1062 (0%)]	Loss: 0.159817
Train Epoch: 6 [1000/1062 (93%)]	Loss: 0.132921
Train Epoch: 7 [0/1062 (0%)]	Loss: 0.155876
Train Epoch: 7 [1000/1062 (93%)]	Loss: 0.136516
Train Epoch: 8 [0/1062 (0%)]	Loss: 0.161397
Train Epoch: 8 [1000/1062 (93%)]	Loss: 0.135818
Train Epoch: 9 [0/1062 (0%)]	Loss: 0.156751
Train Epoch: 9 [1000/1062 (93%)]	Loss: 0.141039
Train Epoch: 10 [0/1062 (0%)]	Loss: 0.158248
Train Epoch: 10 [1000/1062 (93%)]	Loss: 0.133863
torch.save  ./log/MLP32-SST-best-0.475525.pth
[0.4755251]
Train Epoch: 11 [0/1

KeyboardInterrupt: 

# eval

In [11]:
def test(target  ,m  ,model,code_shape,output_shape,device):

    model = model.to(device)
    mask_map,meansure_x,meansure_y = m.grid2D(reqxy=True)
    mask_map = torch.Tensor(mask_map)

    target_re = (target.float()).reshape(output_shape).to(device)
    mask_re   = mask_map.reshape(output_shape).to(device)

    f = model.loss_decoder_helper(target_re,mask_re)
    fp= model.grad_loss_decoder_helper(target_re,mask_re)

    start_code = np.zeros(args.mod_number).astype(np.float32)+0.1
    fmin_code =fmin_cg(f,start_code,fprime=fp,disp=False )
    B_code_shape = [1,]+list(code_shape)
    y_per = model.decode( torch.Tensor(fmin_code.astype(np.float32)).to(device).reshape(B_code_shape))


    DS = (target.flatten().detach().cpu()-y_per.flatten().detach().cpu())
    D = torch.sqrt((DS**2).sum())
    T = torch.sqrt(((target.detach()+TM)**2).sum())
    L2 = D/T
    L2 = L2.cpu().numpy()

    y_per = y_per.flatten().detach().cpu().numpy()


    return y_per,L2,meansure_x,meansure_y

In [12]:
target = val_dataset[0]

mm = Monitor(args.monitorType, args.data_shape,random_num = 20  ,mask = np.load(args.dataset_mask_path))
val_res,L2,meansure_x,meansure_y = test(target - TM ,mm,model,args.code_shape,args.mod_output_shape,args.device)


In [13]:
L2

array(0.03808995, dtype=float32)

In [14]:
import numpy.ma as ma

def plot_save(x,name,xy=None):
    x = ma.masked_array((x).reshape(args.data_shape),SSTmask)

    plt.figure(figsize=(8,6),dpi=300)
    plt.imshow(x,cmap='coolwarm',vmin=0, vmax=32)
    plt.colorbar()
    plt.xticks([])
    plt.yticks([])

    if xy is not None:
        x,y = xy
        plt.scatter(y,x,4,c='g')
    plt.title(name)
    plt.savefig(name+ '.png')
    plt.savefig(name+ '.pdf')
    

In [15]:
from scipy import signal as signal

kernel = np.ones(3,3)

target_c = signal.convolve2d(target,kernel)


plot_save(target_c,'target',xy=mm.xy)

TypeError: Cannot interpret '3' as a data type

In [ ]:
plot_save(val_res+TM.flatten(),'$N_{sensors} = 20$',xy=mm.xy)

# baselne : GPOD